In [1]:
require 'nn';

In [2]:
i = 5
o = 3
d = 4

In [3]:
autoencoder = nn.Sequential()
autoencoder:add(nn.Linear(i,o))
autoencoder:add(nn.Sigmoid())
autoencoder:add(nn.Linear(o,d))
autoencoder:add(nn.Sigmoid());

In [4]:
autoencoder:zeroGradParameters()

In [5]:
inp = torch.randn(5)

In [6]:
out = autoencoder:forward(inp)

In [7]:
autoencoder:backward(inp, torch.ones(4))

0.01 *
-1.1615
-0.7341
-0.1560
 0.5619
-0.3447
[torch.DoubleTensor of size 5]



In [8]:
autoencoder

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> output]
  (1): nn.Linear(5 -> 3)
  (2): nn.Sigmoid
  (3): nn.Linear(3 -> 4)
  (4): nn.Sigmoid
}
{
  gradInput : DoubleTensor - size: 5
  modules : 
    {
      1 : 
        nn.Linear(5 -> 3)
        {
          gradBias : DoubleTensor - size: 3
          weight : DoubleTensor - size: 3x5
          gradWeight : DoubleTensor - size: 3x5
          gradInput : DoubleTensor - size: 5
          bias : DoubleTensor - size: 3
          output : DoubleTensor - size: 3
        }
      2 : 
        nn.Sigmoid
        {
          gradInput : DoubleTensor - size: 3
          output : DoubleTensor - size: 3
        }
      3 : 
        nn.Linear(3 -> 4)
        {
          gradBias : DoubleTensor - size: 4
          weight : DoubleTensor - size: 4x3
          gradWeight : DoubleTensor - size: 4x3
          gradInput : DoubleTensor - size: 3
          bias : DoubleTensor - size: 4
          output : DoubleTensor - size: 4
        }
      4 : 
  

In [9]:
autoencoder.modules[4].gradInput

 0.2475
 0.2051
 0.2403
 0.2449
[torch.DoubleTensor of size 4]



$$\sigma_2'(W'B+b') = A\odot(1-A)\odot\frac{\partial L}{\partial A}$$

In [10]:
torch.cmul(torch.cmul(torch.ones(4)-autoencoder.output, autoencoder.output),torch.ones(4))

 0.2475
 0.2051
 0.2403
 0.2449
[torch.DoubleTensor of size 4]



$$\frac{\partial A}{\partial b'} = \sigma_2'(W'B+b')$$ 

In [11]:
autoencoder.modules[3].gradBias

 0.2475
 0.2051
 0.2403
 0.2449
[torch.DoubleTensor of size 4]



$$\frac{\partial A}{\partial W'} $$

In [12]:
autoencoder.modules[3].gradWeight

 0.1250  0.1315  0.1491
 0.1036  0.1089  0.1235
 0.1213  0.1276  0.1447
 0.1236  0.1301  0.1475
[torch.DoubleTensor of size 4x3]



$$\frac{\partial A}{\partial W'} = \sigma_2'(W'B+b')B^\top$$

In [13]:
autoencoder.modules[4].gradInput:reshape(4,1)*autoencoder.modules[2].output:reshape(1,3)

 0.1250  0.1315  0.1491
 0.1036  0.1089  0.1235
 0.1213  0.1276  0.1447
 0.1236  0.1301  0.1475
[torch.DoubleTensor of size 4x3]



$$\frac{\partial A}{\partial B}$$

In [14]:
autoencoder.modules[3].gradInput

0.01 *
 4.4624
 9.7010
-0.8921
[torch.DoubleTensor of size 3]



$$\frac{\partial A}{\partial B} = W^\top \sigma_2'(W'B+b')$$

In [15]:
autoencoder.modules[3].weight:transpose(1,2)*autoencoder.modules[4].gradInput:reshape(4,1)

0.01 *
  4.4624
  9.7010
 -0.8921
[torch.DoubleTensor of size 3x1]



$$\sigma_1'(Wx+b) = B\odot(1-B)\odot\frac{\partial A}{\partial B}$$

In [16]:
torch.cmul(torch.cmul(torch.ones(3)-autoencoder.modules[2].output, autoencoder.modules[2].output),autoencoder.modules[3].gradInput)

0.01 *
 1.1155
 2.4158
-0.2137
[torch.DoubleTensor of size 3]



$$\frac{\partial B}{\partial b} = \sigma_1'(Wx+b)$$

In [17]:
autoencoder.modules[1].gradBias

0.01 *
 1.1155
 2.4158
-0.2137
[torch.DoubleTensor of size 3]



$$\frac{\partial B}{\partial W}$$

In [18]:
autoencoder.modules[1].gradWeight

0.01 *
 -1.7765 -0.5530 -1.2678 -1.4382  0.0477
 -3.8475 -1.1976 -2.7457 -3.1147  0.1033
  0.3404  0.1059  0.2429  0.2755 -0.0091
[torch.DoubleTensor of size 3x5]



$$\frac{\partial B}{\partial W} = \sigma_1'(Wx+b)x^\top$$

In [19]:
autoencoder.modules[2].gradInput:reshape(3,1)*inp:reshape(1,5)

0.01 *
 -1.7765 -0.5530 -1.2678 -1.4382  0.0477
 -3.8475 -1.1976 -2.7457 -3.1147  0.1033
  0.3404  0.1059  0.2429  0.2755 -0.0091
[torch.DoubleTensor of size 3x5]

